# Species wrangling
In this notebook, I'll combine some information about which species we're interested in using for this project (from manually created spreadsheets) and find sources for genomes/transcriptomes for those we don't already have.

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, exists
from openpyxl import load_workbook
from collections import defaultdict, Counter
import numpy as np
from pysradb.sraweb import SRAweb
from tqdm import tqdm
import subprocess

/mnt/home/lotrecks/.conda/envs/omics/lib/python3.10/site-packages/pysradb/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [38]:
run_date = '18Sep2025'

## Reading in the data

In [13]:
phyto_origin = pd.read_csv('../data/species_info/species_info_full.csv')
phyto_origin.head()

,abbreviated_species,species_gbif,species,common_name,has_genome,has_transcriptome,original_source,thermotolerance,reference,notes
0,A. americanus,Acorus americanus,Acorus americanus,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
1,A. tequilanavar,Agave tequilana,Agave tequilana,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
2,A. officinalis,Althaea officinalis,Althaea officinalis,marsh-mallow,NaN,NaN,phytozome,NaN,NaN,NaN
3,A. linifolium,Alyssum linifolium,Alyssum linifolium,NaN,NaN,NaN,phytozome,NaN,NaN,NaN
4,A. hypochondriacus,Amaranthus hypochondriacus,Amaranthus hypochondriacus,NaN,NaN,NaN,phytozome,NaN,NaN,NaN


Add a path for the genomes we already have:

In [14]:
genome_top_path = '/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/phytozome_genomes'
sp_path = [(f"{f.split('_')[0][0]}. {f.split('_')[0][1:]}", f'{genome_top_path}/{f}') for f in listdir(genome_top_path)]
sp_path_df = pd.DataFrame(sp_path, columns=['abbreviated_species', 'full_path'])
phyto_paths = sp_path_df.merge(phyto_origin, left_on='abbreviated_species', right_on='abbreviated_species', how='right')

Include function to read URLs from Excel files:

In [15]:
def get_hyperlink_columns(base_df, path, sheet_name):
    """
    Use openpyxl to detect hyperlinks and add them as new columns.

    From: https://github.com/serenalotreck/greencut-formatting/blob/main/notebooks/greencut_database_input_formatting.ipynb

    parameters:
        base_df, pandas df: the result of read_excel on path
        path, str: path to the excel document
        sheet_name, str: name of the sheet to read

    returns:
        updated_df, pandas df: base_df with added columns for hyperlinks
    """
    # Load the workbook using openpyxl
    workbook = load_workbook(path)
    
    # Select the sheet
    sheet = workbook[sheet_name] if sheet_name else workbook.active

    # Openpyxl will maintain rows that aren't relevant to the dataframe,
    # whereas pandas.read_excel just drops those -- we need to make them agree
    if sheet.max_row != len(base_df):
        rows_to_delete = [i for i in range(len(base_df) + 1, sheet.max_row)]
        for row_index in reversed(rows_to_delete):
            sheet.delete_rows(row_index)

    # Detect columns with hyperlinks
    hyperlink_cols = defaultdict(list)
    for row_idx, row in enumerate(sheet.iter_rows()):
        # Skip the first row, since it's the header
        if row_idx == 0:
            continue
        for col_index, cell in enumerate(row):
            hyperlink = cell.hyperlink.target if cell.hyperlink else None
            header_name = sheet.cell(row=1, column=col_index + 1).value
            if hyperlink:
                hyperlink_cols[header_name + "_hyperlink"].append(hyperlink)
            else:
                # We need to add NaN if there's no hyperlink to keep the values
                # in the right rows, but this does mean we'll get non-hyperlink columns
                hyperlink_cols[header_name + "_hyperlink"].append(np.nan)

    # Create a df with the new columns
    hyperlink_df = pd.DataFrame(hyperlink_cols)
    
    # Now remove any columns that are all NaN
    hyperlink_df = hyperlink_df.dropna(axis=1, how='all')

    # Concat this horizontally to the original df
    updated_df = pd.concat([base_df, hyperlink_df], axis=1)

    return updated_df

In [16]:
with_lit = pd.read_excel('../data/species_info/thermophilic_plant_species.xlsx', sheet_name='Sheet1')
with_lit.columns = with_lit.columns.str.replace(' ', '_')
with_list = with_lit.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
with_lit.head()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes
0,Heliotropium curisavvicum,a prostrate plant from the Mojave desert. You...,NaN,NaN,Rowan suggested
1,Sesuvium verrucosum,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested
2,Sesuvium trianthema,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested
3,Ipomea pes-caprae,"tropical beach vine, grows on hot sands of tro...",NaN,NaN,Rowan suggested
4,Mollugo verticillata,carpet weed. Grows across blacktop that burns...,NaN,NaN,Rowan suggested


## Wrangling
Drop specific rows we know don't have info from manually looking:

In [17]:
with_lit = with_lit.drop(index=[60,125])
with_lit.tail()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes
120,Azorella trifurcata (Gaertn.) Pers.,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
121,Azorella trisecta (H.Wolff) Mart.Fernández & C...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
122,Azorella ulicina (Gillies & Hook.) G.M.Plunket...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
123,Azorella valentini (Speg.) Mart.Fernández & C....,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t..."
124,Cistanthe longiscapa,CAM species that grows in the atacama desert,NaN,NaN,"Paper from Ariel Orellana, gave a talk in the ..."


In [18]:
lit_w_links = get_hyperlink_columns(with_lit, '../data/species_info/thermophilic_plant_species.xlsx', sheet_name='Sheet1').drop(index=60)
lit_w_links.tail()

,Species_Name,Species_Notes,Transciptomic_data_in_NCBI_with_SRA_data,Publication_Link,Notes,Species Name_hyperlink
120,Azorella trifurcata (Gaertn.) Pers.,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
121,Azorella trisecta (H.Wolff) Mart.Fernández & C...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
122,Azorella ulicina (Gillies & Hook.) G.M.Plunket...,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",https://en.wikipedia.org/w/index.php?title=Azo...
123,Azorella valentini (Speg.) Mart.Fernández & C....,NaN,NaN,NaN,"Grows in the high Andes, super arid, appears t...",NaN
124,Cistanthe longiscapa,CAM species that grows in the atacama desert,NaN,NaN,"Paper from Ariel Orellana, gave a talk in the ...",NaN


Get rid of additional text after the first two space-delimited words in the species column to get a base species name.

In [19]:
lit_w_links['species'] = lit_w_links['Species_Name'].str.split(r'[\xa0\s]+').str[:2].str.join(' ')

In [20]:
lit_w_links = lit_w_links.rename(columns={'Notes': 'original_source', 'Species_Notes': 'notes', 'Publication_Link': 'reference'})
lit_w_links.head()

,Species_Name,notes,Transciptomic_data_in_NCBI_with_SRA_data,reference,original_source,Species Name_hyperlink,species
0,Heliotropium curisavvicum,a prostrate plant from the Mojave desert. You...,NaN,NaN,Rowan suggested,NaN,Heliotropium curisavvicum
1,Sesuvium verrucosum,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested,NaN,Sesuvium verrucosum
2,Sesuvium trianthema,Succulent halophytes from deserts and hot coas...,NaN,NaN,Rowan suggested,NaN,Sesuvium trianthema
3,Ipomea pes-caprae,"tropical beach vine, grows on hot sands of tro...",NaN,NaN,Rowan suggested,NaN,Ipomea pes-caprae
4,Mollugo verticillata,carpet weed. Grows across blacktop that burns...,NaN,NaN,Rowan suggested,NaN,Mollugo verticillata


Merge with the other list:

In [21]:
phyto_paths['reference'] = phyto_paths['reference'].astype('object')
all_species = lit_w_links.merge(phyto_paths, left_on=['species', 'notes', 'reference'], right_on=['species', 'notes', 'reference'], suffixes=('_berkley', '_serena'), how='outer')
all_species = all_species.drop(columns=['Species_Name', 'Transciptomic_data_in_NCBI_with_SRA_data', 'Species Name_hyperlink', 'abbreviated_species'])
all_species = all_species[['species', 'thermotolerance'] + sorted(c for c in all_species.columns if c != 'species' and c != 'thermotolerance')]
all_species = all_species.drop_duplicates()
all_species.head()

,species,thermotolerance,common_name,full_path,has_genome,has_transcriptome,notes,original_source_berkley,original_source_serena,reference,species_gbif
0,Acorus americanus,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Acorus americanus
1,Agave sp.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agave tequilana,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Agave tequilana
3,Althaea officinalis,NaN,marsh-mallow,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Althaea officinalis
4,Alyssum linifolium,NaN,NaN,/mnt/research/Walker_Lab_Research/Serena_proje...,NaN,NaN,NaN,NaN,phytozome,NaN,Alyssum linifolium


How many genera are represented here?

In [22]:
unique_genera = all_species.species.str.split(' ').str[0].value_counts()
print(f'There are {len(unique_genera)} unique genera in the list. The top genus is {unique_genera.idxmax()}, with {unique_genera.max()} species')
spec_in_common = set(phyto_paths.species).intersection(set(lit_w_links.species))
print(f'There were only {len(spec_in_common)} species in common between the two lists: {", ".join(list(spec_in_common))}')

There are 159 unique genera in the list. The top genus is Azorella, with 58 species
There were only 14 species in common between the two lists: Boechera crandallii, Boechera gunnisoniana, Salvadora persica, Boechera depauperata, Haloxylon salicornicum, Boechera pallidifolia, Boechera perennans, Vigna unguiculata, Calligonum comosum, Avicennia marina, Boechera holboellii, Salsola villosa, Phoenix dactylifera, Boechera stricta


Save the final thing:

In [23]:
all_species.to_csv(f'../data/species_info/berkley_serena_combined_species_list_{run_date}.csv', index=False)

## Looking in SRA
We want to know, before we manually search up all of these, that there's at least one entry in SRA for them. We'll use [`pysradb`](https://github.com/saketkc/pysradb) for this. We also can only use RNA-Seq data, so we'll also filter for that here.

In [24]:
db = SRAweb()

In [104]:
all_studies = {}
missing_rnaseq = []
for spec in tqdm(all_species.species):
    try:
        if isfile(all_species.loc[all_species.species==spec, 'full_path'].values[0]):
            print(f'Skipping species {spec} because it has a genome in Phytozome')
            continue
    except TypeError:
        studies = db.search_sra(spec, detailed=True)
        try:
            studies_rna = studies[studies.library_strategy.str.contains('RNA')]
            if len(studies_rna) > 0:
                print(f'There are {len(studies_rna)} RNA-seq studies in SRA for species {spec}')
                all_studies[spec] = studies_rna
            else:
                print(f'0 RNA-seq studies found for species {spec}.')
                missing_rnaseq.append(spec)
        except AttributeError:
            print(f'0 RNA-seq studies found for species {spec}.')
            missing_rnaseq.append(spec)

  0%|          | 0/294 [00:00<?, ?it/s]

Skipping species Acorus americanus because it has a genome in Phytozome


  1%|          | 2/294 [00:03<09:39,  1.99s/it]

0 RNA-seq studies found for species Agave sp..
Skipping species Agave tequilana  because it has a genome in Phytozome
Skipping species Althaea officinalis because it has a genome in Phytozome
Skipping species Alyssum linifolium because it has a genome in Phytozome
Skipping species Amaranthus hypochondriacus because it has a genome in Phytozome
Skipping species Amborella trichopoda because it has a genome in Phytozome
Skipping species Anacardium occidentale because it has a genome in Phytozome
Skipping species Ananas comosus because it has a genome in Phytozome
Skipping species Andropogon gerardi because it has a genome in Phytozome
Skipping species Aquilegia caerulea because it has a genome in Phytozome
Skipping species Arabidopsis halleri because it has a genome in Phytozome
Skipping species Arabidopsis lyrata because it has a genome in Phytozome
Skipping species Arabidopsis thaliana because it has a genome in Phytozome
Skipping species Arachis hypogaea because it has a genome in Phyt

  5%|▌         | 16/294 [00:33<09:44,  2.10s/it]

...

There are 58 RNA-seq studies in SRA for species Yucca brevifolia
Skipping species Zea mays because it has a genome in Phytozome


 99%|█████████▉| 292/294 [11:55<00:03,  1.52s/it]

There are 36 RNA-seq studies in SRA for species Ziziphus spina


100%|██████████| 294/294 [12:00<00:00,  2.45s/it]

There are 49 RNA-seq studies in SRA for species Ziziphus spina christi
Skipping species Zostera marina because it has a genome in Phytozome


In [109]:
print(f'There are {len(all_studies)} organisms with RNAseq datasets that also don\'t have Phytozome genomes, {len(list(listdir(genome_top_path)))} organisms with Phytozome genomes, and {len(missing_rnaseq)} that don\'t have an RNAseq dataset in SRA.')

There are 32 organisms with RNAseq datasets that also don't have Phytozome genomes, 162 organisms with Phytozome genomes, and 101 that don't have an RNAseq dataset in SRA.


In [110]:
print([len(stud) for stud in all_studies.values()], sum([len(stud) for stud in all_studies.values()]))

[127, 1, 6, 16, 124, 3, 12, 5, 3, 3, 124, 11, 3, 3, 12, 5, 18, 5, 16, 16, 1, 2, 6, 340, 157, 11, 2, 1, 1, 58, 36, 49] 1177


Concat so that we can just delete the studies we don't want from a single spreadsheet:

In [111]:
for i, reps in enumerate([len(df.columns) - len(set(df.columns)) for df in all_studies.values()]):
    if reps > 0:
        print(f'Species {list(all_studies.keys())[i]} detailed metadata has duplicate columns')
        species_df = list(all_studies.values())[i]
        # Get the duplicated columns and their counts
        counted_col_names = Counter(species_df.columns.tolist())
        dup_cols = [k for k,v in counted_col_names.items() if v > 1]
        # Check which version of the columns actually have data (if any)
        dup_col_isna = [(col_name, [vals.isna().all() for col, vals in species_df[col_name].items()]) for col_name in dup_cols]
        # If they're all na, drop them all, if  not, keep the ones with info
        # In our dataset there's only one with data in every duplicated group, you'd want to
        # add a secondary check here if you were adding different species
        for col_name, contents in dup_col_isna:
            if set(contents) == set([True]):
                species_df = species_df.drop(columns=col_name)
                all_studies[list(all_studies.keys())[i]] = species_df
            else:
                # Get unique cols
                unique_cols = species_df.columns.duplicated(keep=False) == False
                df_unique = species_df.loc[:, unique_cols]
                # Choose the duplicated column that actually has data
                col_to_keep = species_df.T.groupby(species_df.T.index, axis=0).nth(contents.index(False)).T
                species_df = pd.concat([df_unique, col_to_keep], axis=1)
                all_studies[list(all_studies.keys())[i]] = species_df

In [112]:
all_studies_df = pd.concat([df.reset_index(drop=True) for df in all_studies.values()], ignore_index=True)#.drop_duplicates()
all_studies_df.head()

,run_accession,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,...,subsrc_note,authority,library name,sample type,geo_loc,plant_id,geographic location,time,source population,tag
0,DRR119672,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,82927,Avicennia marina,<NA>,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR10533306,SRP233272,Tissue-specific transcriptomes from Avicennia ...,SRX7217273,Tissue-specific transcriptomes from Avicennia ...,Tissue-specific transcriptomes from Avicennia ...,82927,Avicennia marina,Transcriptomic reads of Avicennia_marina_Flowers,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR12145926,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_Leaf_salt_0h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR12145992,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666716,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_24h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR12145993,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666717,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_48h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Save out the studies so we don't have to run the search again:

In [113]:
all_studies_df.to_csv(f'/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/all_SRA_studies_{run_date}.csv', index=False)

Read them back in:

In [83]:
all_studies_df = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/all_SRA_studies_17Sep2025.csv')
all_studies_df.head()

,run_accession,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,...,subsrc_note,authority,library name,sample type,geo_loc,plant_id,geographic location,time,source population,tag
0,DRR119672,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,82927,Avicennia marina,NaN,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR10533306,SRP233272,Tissue-specific transcriptomes from Avicennia ...,SRX7217273,Tissue-specific transcriptomes from Avicennia ...,Tissue-specific transcriptomes from Avicennia ...,82927,Avicennia marina,Transcriptomic reads of Avicennia_marina_Flowers,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR12145926,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_Leaf_salt_0h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR12145992,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666716,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_24h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR12145993,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666717,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_salt_leaf_48h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We want to perform further filtering on this list to get a single rep per organism. This will require some if/else logic on the studies for each organism. First, we'll drop any `SINGLE` in the `library_layout` column, if there is a `PAIRED` option for that species, because paired end reads are more reliable.

In [84]:
drop_for_single = []
for species in all_studies_df.organism_name.unique():
    spec_subset = all_studies_df[all_studies_df.organism_name == species]
    single_end = spec_subset[spec_subset.library_layout == 'SINGLE']
    if len(single_end) == len(spec_subset): # Means there's only single end
        print(f'There are only single-end reads for species {species}')
        continue
    else:
        drop_for_single.extend(single_end.index.tolist())
        if len(single_end) > 0:
            print(f'For species {species}, {len(single_end)} of {len(spec_subset)} total studies are single-end reads, and will be dropped')

For species Avicennia marina, 5 of 88 total studies are single-end reads, and will be dropped
There are only single-end reads for species Boechera depauperata
For species Boechera divaricarpa, 15 of 48 total studies are single-end reads, and will be dropped
For species Boechera stricta, 49 of 115 total studies are single-end reads, and will be dropped
There are only single-end reads for species Arabidopsis thaliana
For species Boechera williamsii, 18 of 36 total studies are single-end reads, and will be dropped
For species Boechera pallidifolia, 24 of 33 total studies are single-end reads, and will be dropped
There are only single-end reads for species Opuntia robusta
There are only single-end reads for species Opuntia joconostle
There are only single-end reads for species Opuntia ficus-indica
For species Phoenix dactylifera, 23 of 267 total studies are single-end reads, and will be dropped
There are only single-end reads for species Coccotrypes dactyliperda
There are only single-end r

In [85]:
all_studies_df = all_studies_df.drop(index=drop_for_single)

Unfortunately the read length is not included in the metadata returned by our search, so we can't filter based on that; we'll have to check that manually at the end. However, we also want to know that our samples are coming from leaf tissue. While there is not a eddicated field for tissue type, many studies inclued this information in one of the following fields: `study_title`, `experiment_title`, `experiment_desc`, `tissue`, `tissue_type`. As for the single vs. paired filter, we'll look for any rows with the string `"leaf"` in them, but if none have them, we'll just keep what's there.

In [86]:
drop_for_leaf = []
for species in all_studies_df.organism_name.unique():
    spec_subset = all_studies_df[all_studies_df.organism_name == species]
    # This feels unwieldy and like there's probably a cleaner way to do it
    has_leaf = spec_subset[(spec_subset.study_title.str.lower().str.contains('leaf')) | (spec_subset.experiment_title.str.lower().str.contains('leaf')) | (spec_subset.experiment_desc.str.lower().str.contains('leaf')) | (spec_subset.tissue.str.lower().str.contains('leaf')) | (spec_subset.tissue_type.str.lower().str.contains('leaf')) | (spec_subset.study_title.str.lower().str.contains('leaves')) | (spec_subset.experiment_title.str.lower().str.contains('leaves')) | (spec_subset.experiment_desc.str.lower().str.contains('leaves')) | (spec_subset.tissue.str.lower().str.contains('leaves')) | (spec_subset.tissue_type.str.lower().str.contains('leaves'))]
    if len(has_leaf) == 0: # Means there's no leaf specific one
        print(f'There are no leaf-specific studies for species {species}')
        continue
    else:
        not_leaf_idxs = spec_subset.drop(index=has_leaf.index.tolist()).index.tolist()
        drop_for_leaf.extend(not_leaf_idxs)
        print(f'For species {species}, {len(has_leaf)} of {len(spec_subset)} total studies are leaf specific, all others will be dropped')

For species Avicennia marina, 50 of 83 total studies are leaf specific, all others will be dropped
There are no leaf-specific studies for species mangrove metagenome
For species Rhizophora apiculata, 6 of 12 total studies are leaf specific, all others will be dropped
For species Avicennia marina var. rumphiana, 2 of 2 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. eucalyptifolia, 1 of 2 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. australasica, 1 of 1 total studies are leaf specific, all others will be dropped
For species Avicennia marina subsp. marina, 1 of 1 total studies are leaf specific, all others will be dropped
There are no leaf-specific studies for species sediment metagenome
There are no leaf-specific studies for species Azorella trisecta
There are no leaf-specific studies for species Boechera depauperata
There are no leaf-specific studies for species Boechera divaricarpa
There

In [87]:
all_studies_df = all_studies_df.drop(index=drop_for_leaf)

The last thing we can do tom ake this list easiest to manage is to remove duplicate entries from the same experiment, since we only need one rep. Any species that still have multiple experiments we will include one rep from all of them, as we can manually check the read lengths to choose among them. The groupings will have the same Bioproject number, but different sample accession numbers and different experiment numbers. We'll go through each remaining bioproject and keep just the first sample. Note that some bioprojects contain multiple species, so we're first going to look at each species and then within that species, the bioproject numbers.

In [93]:
drop_for_reps = []
for species in all_studies_df.organism_name.unique():
    spec_subset = all_studies_df[all_studies_df.organism_name == species]
    for proj in spec_subset.bioproject.unique():
        bio_subset = spec_subset[spec_subset.bioproject == proj]
        all_but_first = bio_subset.index.tolist()[1:]
        drop_for_reps.extend(all_but_first)
print(f'{len(drop_for_reps)} replicate rows will be dropped (out of a total of {len(all_studies_df)})')

586 replicate rows will be dropped (out of a total of 684)


In [95]:
all_studies_df = all_studies_df.drop(index=drop_for_reps)

Save out what we have:

In [98]:
all_studies_df.to_csv(f'/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_{run_date}.csv', index=False)

Since I revised the above code to use the `detailed` additional columns, I already have the read lengths for some but not all of the studies that result from adding the `tissue` and `tissue_type` columns. We'll merge the old results here to save the work of having to manually add them all back: 

In [99]:
all_studies_df = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_18Sep2025.csv')
all_studies_df.head()

,run_accession,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,...,subsrc_note,authority,library name,sample type,geo_loc,plant_id,geographic location,time,source population,tag
0,DRR119672,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,82927,Avicennia marina,NaN,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR10533306,SRP233272,Tissue-specific transcriptomes from Avicennia ...,SRX7217273,Tissue-specific transcriptomes from Avicennia ...,Tissue-specific transcriptomes from Avicennia ...,82927,Avicennia marina,Transcriptomic reads of Avicennia_marina_Flowers,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR12145926,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Avicennia marina,am_Leaf_salt_0h,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR12146026,SRP270005,Whole Transcriptome sequencing and analaysis i...,SRX8666750,RNASeq from Avicennia leaf tissue,RNASeq from Avicennia leaf tissue,82927,Avicennia marina,am_rnaseq_leaf,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR12185868,SRP271133,Gazi bay and Mida creek mangrove metagenome Ra...,SRX8700441,16S rRNA amplicon metagenomics sequences from ...,16S rRNA amplicon metagenomics sequences from ...,1284368,mangrove metagenome,Ed16S-AVM1-1-5cm,RNA-Seq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
with_read_lens = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_15Sep2025_with_read_lengths.csv')
with_read_lens = with_read_lens.drop(columns='Unnamed: 8')
with_read_lens.head()

,organism_name,len_reads,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,library_name,library_strategy,...,biosample,bioproject,instrument,instrument_model,instrument_model_desc,total_spots,total_size,run_accession,run_total_spots,run_total_bases
0,Adenophora,150,SRP406394,Phylogenetic Genomics of Adenophora,SRX18176188,RNA-seq of Adenophora stricta subsp. stricta :,RNA-seq of Adenophora stricta subsp. stricta :,82270,am_Leaf_salt_0h,RNA-Seq,...,SAMN15447932,PRJNA643813,NextSeq 500,NextSeq 500,ILLUMINA,11648210,757097881,SRR12145926,11648210,1750406459
1,Adenophora stricta,150,SRP549143,This BioProject includes raw sequencing data f...,SRX26957374,leaf+stem RNA of Adenophora stricta,leaf+stem RNA of Adenophora stricta,82279,am_rnaseq_leaf,RNA-Seq,...,SAMN15447991,PRJNA392014,NextSeq 500,NextSeq 500,ILLUMINA,54006517,6346122539,SRR12146026,54006517,14987554557
2,Avicennia marina,75,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,82927,Ed16S-AVM1-1-5cm,RNA-Seq,...,SAMN15492854,PRJNA644929,Illumina MiSeq,Illumina MiSeq,ILLUMINA,49648,17902988,SRR12185868,49648,29735410
3,Avicennia marina,139,SRP270005,Whole Transcriptome sequencing and analaysis i...,SRX8666750,RNASeq from Avicennia leaf tissue,RNASeq from Avicennia leaf tissue,82927,RA_leaf_500_r2,RNA-Seq,...,SAMN22185018,PRJNA719266,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,26620432,3597436477,SRR16279084,26620432,5270845536
4,Avicennia marina,76,SRP058424,Transcriptomic changes in Avicennia marina in...,SRX1030223,Avicennia marina A1 accession control leaf tra...,Avicennia marina A1 accession control leaf tra...,82927,Arumphiana_leaf,RNA-Seq,...,SAMN26804911,PRJNA817364,Illumina NovaSeq 6000,Illumina NovaSeq 6000,ILLUMINA,30063634,3007376940,SRR18449582,30063634,9019090200


Because we had additional fields to filter on, the newer data frame will have some studies dropped compared to the old if having the additional fields identified leaf-specific studies. The symptom of this in the merged dataframe is rows that have a run accession but no other information; we'll drop any rows that have no organism name.

In [101]:
# Fix the type of this one col that causes an issue
all_studies_df.sample_title = all_studies_df.sample_title.astype('object')
# Merge
merged = all_studies_df.merge(with_read_lens, left_on='run_accession', right_on='run_accession', suffixes=('_new', '_old'), how='outer')
# Drop cols from old
merged = merged.drop(columns=[col for col in merged.columns if col[-4:] == '_old'])
# Drop _new suffix
merged.columns = merged.columns.str.replace('_new', '', regex=False)
# Drop rows with no info in the organism name column
merged = merged.dropna(subset=['organism_name'])

In [103]:
# Make the first three columns be the species, the run accession, and the read length
merged = merged[['organism_name', 'run_accession', 'len_reads', 'tissue', 'tissue_type'] + [col for col in merged.columns if col not in ['organism_name', 'run_accession', 'len_reads', 'tissue', 'tissue_type']]]
merged.head()

,organism_name,run_accession,len_reads,tissue,tissue_type,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,...,subsrc_note,authority,library name,sample type,geo_loc,plant_id,geographic location,time,source population,tag
0,Avicennia marina,DRR119672,NaN,NaN,leaf,DRP004551,Avicennia marina leaves transcriptome,DRX112674,Illumina HiSeq 1500 paired end sequencing of S...,Illumina HiSeq 1500 paired end sequencing of S...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Boechera gunnisoniana,ERR1548596,101,NaN,NaN,ERP016601,Abiotic stress-response reprogramming events u...,ERX1619366,Illumina HiSeq 2000 paired end sequencing; Abi...,Illumina HiSeq 2000 paired end sequencing; Abi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sesuvium verrucosum,ERR2040195,NaN,NaN,NaN,ERP023948,1000 Plant (1KP) Transcriptomes: The Remaining...,ERX2099252,Illumina HiSeq 2000 paired end sequencing,Illumina HiSeq 2000 paired end sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Trigastrotheca pentaphylla,ERR2040238,NaN,NaN,NaN,ERP023948,1000 Plant (1KP) Transcriptomes: The Remaining...,ERX2099295,Illumina HiSeq 2000 paired end sequencing,Illumina HiSeq 2000 paired end sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mollugo verticillata,ERR2040239,NaN,NaN,NaN,ERP023948,1000 Plant (1KP) Transcriptomes: The Remaining...,ERX2099296,Illumina HiSeq 2000 paired end sequencing,Illumina HiSeq 2000 paired end sequencing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
merged.to_csv(f'/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_{run_date}_with_len_reads.csv', index=False)

## Getting fastq files
After exporting the above file, I manually checked read lengths and removed most duplicate samples. For those with similar length paired read samples, I left duplicates if (a) I thought it would be valuable or (b) if I wasn't certain about the tissue origin for the samples. Here, we'll read back in the manually finalized dataframe and use it to download the datasets.

In [105]:
final_df = pd.read_csv('/mnt/research/Walker_Lab_Research/Serena_project_data/selection-under-heat_data/sra_study_info/filtered_single_rep_studies_18Sep2025_manual_filtering.csv')
final_df.head()

,organism_name,run_accession,len_reads,tissue,tissue_type,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,...,subsrc_note,authority,library name,sample type,geo_loc,plant_id,geographic location,time,source population,tag
0,Adenophora stricta,SRR31592529,150,leaf-stem,NaN,SRP549143,This BioProject includes raw sequencing data f...,SRX26957374,leaf+stem RNA of Adenophora stricta,leaf+stem RNA of Adenophora stricta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Avicennia marina,SRR12145926,150,Leaf,NaN,SRP270003,RNASeq to identify salt responsive genes from ...,SRX8666650,RNASeq to identify salt responsive genes from ...,RNASeq to identify salt responsive genes from ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Avicennia marina,SRR12146026,150,Leaf,NaN,SRP270005,Whole Transcriptome sequencing and analaysis i...,SRX8666750,RNASeq from Avicennia leaf tissue,RNASeq from Avicennia leaf tissue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Avicennia marina subsp. australasica,SRR18449588,100,leaf,NaN,SRP365449,Mangrove Genome Project,SRX14582333,Amarinaau_leaf,Amarinaau_leaf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Avicennia marina subsp. eucalyptifolia,SRR18449587,100,leaf,NaN,SRP365449,Mangrove Genome Project,SRX14582334,Amarinaeu_leaf,Amarinaeu_leaf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rather than use the SRA toolkit, which is prohibitively slow, we'll use `pysradb` with the `detailed` option to get FTP links for the data we're interested in.

In [107]:
db = SRAweb()

In [108]:
ftp_links = {}


0    leaf-stem
Name: tissue, dtype: object
0    Leaf
Name: tissue, dtype: object
0    Leaf
Name: tissue, dtype: object


KeyboardInterrupt: 